In [54]:
import os
import json
import glob
import shutil

# 기본 경로 설정
base_path = "./cat_center/" 
json_files = glob.glob(os.path.join(base_path, "*.json"))

# 저장 경로 설정
yolo_json_path = os.path.join(base_path, "yolo_json")
yolo_image_path = os.path.join(base_path, "yolo_image")

# 폴더가 없으면 생성
os.makedirs(yolo_json_path, exist_ok=True)
os.makedirs(yolo_image_path, exist_ok=True)

def convert_to_custom_yolo(data, image_width, image_height):
    center = None
    keypoints = []

    for shape in data['shapes']:
        if shape['label'] == 'center':
            x1, y1 = shape['points'][0]
            x2, y2 = shape['points'][1]
            center_x = (x1+(x2-x1)/2)/ image_width
            center_y = (y1+(y2-y1)/2)/ image_height
            width = (x2 - x1) / image_width
            height = (y2 - y1) / image_height
            center = [center_x, center_y, width, height]
        
        elif shape['shape_type'] == 'point':
            x, y = shape['points'][0]
            normalized_x = x / image_width
            normalized_y = y / image_height
            group_id = shape.get('group_id', 0)  # group_id가 없으면 0으로 설정
            
            if group_id == 1:
                group_id = 0
            
            keypoints.append((normalized_x, normalized_y, shape['label'], group_id))
    
    # 키포인트를 라벨 순으로 정렬
    keypoints.sort(key=lambda k: int(k[2]))
    
    # 결과 문자열 생성
    result = f"0 {center[0]} {center[1]} {center[2]} {center[3]}" # box
    for kp in keypoints:
        result += f" {kp[0]} {kp[1]} {kp[3]}" # point
    
    return result

for json_file in json_files:
    with open(json_file) as json_data:
        data = json.load(json_data)
        filename = os.path.basename(json_file)
        image_filename = os.path.splitext(filename)[0] + ".png"  # JSON 파일명을 기반으로 이미지 파일명 추정
        image_width = data.get('imageWidth', 1)  # 이미지 너비가 없을 경우 기본값 1 사용
        image_height = data.get('imageHeight', 1)  # 이미지 높이가 없을 경우 기본값 1 사용
        
        # YOLO 텍스트 파일 저장
        yolo_txt_file = os.path.join(yolo_json_path, os.path.splitext(filename)[0] + ".txt")
        yolo_data = convert_to_custom_yolo(data, image_width, image_height)
        with open(yolo_txt_file, 'w') as f:
            f.write(yolo_data)
        
        # 이미지 복사
        source_image_path = os.path.join(base_path, image_filename)
        dest_image_path = os.path.join(yolo_image_path, image_filename)

        # 폴더로 이동
        if os.path.exists(source_image_path):
            shutil.copy(source_image_path, dest_image_path)
        else:
            print(f"Warning: Image file not found - {source_image_path}")


In [50]:
filename

'TCENTER_002447_011xxxxxxxxxxxx_N_0.json'

In [36]:
print(f"Working directory: {os.getcwd()}")
print(f"Base path: {base_path}")
print(f"JSON files found: {len(json_files)}")

Working directory: /RnD
Base path: ./cat_center/
JSON files found: 512


In [40]:
# 기본 경로 설정
base_path = "./cat_center/" 
json_files = glob.glob(os.path.join(base_path, "*.json"))

# 저장 경로 설정
yolo_json_path = os.path.join(base_path, "yolo_json")
yolo_image_path = os.path.join(base_path, "yolo_image")


'TCENTER_005255_000xxxxxxxxxxxx_N_2.json'

In [5]:
import os, json, glob, shutil
from tqdm import tqdm

# center=["101","102","103","104","201","202","203","204","301","302","303","304","401","402","403","404"]
side = ["U1","U2","U3","U4","U5","U6","U7","U8","D1","D2","D3","D4","D5","D6","D7","D8","D9"]

json_path = "/RnD/dog_yolo/datasets/train/images/" # 모든 jason파일 불러오는 것으로 변경
save_path = glob.glob(os.path.join("/RnD/dog_yolo/datasets/train/images/", "*.json"))
save_img = True
label_sequence = side
for i in json_path:
    with open(i) as json_data:
        data = json.load(json_data)
        filename = data['imagePath']
        h, w = data['imageHeight'], data['imageWidth']
        
        image_path = base_path + filename
        annotation_dir_path = os.path.join(save_path, 'labels', save_as)
        if os.path.exists(annotation_dir_path)==False:
            os.makedirs(annotation_dir_path)
        save_anno = os.path.join(yolo_json_path, os.path.splitext(filename)[0] + ".txt")
        
        # if save_img:
        #     image_dir_path = "/RnD/dog_yolo/datasets/train/labels/"
        #     if os.path.exists(image_dir_path)==False:
        #         os.makedirs(image_dir_path)
        #     save_img = image_dir_path+"/"+filename
        #     shutil.copy(image_path, save_img)
        
        keypoint_data = {label: (0,0,0) for label in label_sequence}

        cx, cy, width, heihgt = 0, 0, 0, 0
        for shape in data['shapes']:
            label = shape['label']
            x, y = shape['points'][0]
            if label == 'side' or label == 'dog' or label == 'tooth':
                x1, y1 = shape['points'][0]
                x2, y2 = shape['points'][1]
                width = (x2-x1) / w
                height = (y2-y1) / h
                cx = (x1+(x2-x1)/2)/w
                cy = (y1+(y2-y1)/2)/h
            elif label in label_sequence:            
                x1 = x / w
                y1 = y / h
                category = shape['group_id']
                keypoint_data[label]=(x1,y1,category)
        yolo_annotation_line = f"{0} {cx} {cy} {width} {height}"
        for label in label_sequence:
            x, y, visible = keypoint_data[label]
            yolo_annotation_line += f" {x} {y} {visible}"

        with open(save_anno, 'w') as f:
            f.write(yolo_annotation_line + "\n")
    

IsADirectoryError: [Errno 21] Is a directory: '/'

In [8]:
import os
import json
import glob
import shutil
from tqdm import tqdm

# 경로 설정
base_path = "/RnD/dog_yolo/datasets/val/"
json_path = os.path.join(base_path, "images", "*.json")
save_path = os.path.join(base_path, "labels")

# 레이블 시퀀스 설정
side = ["U1","U2","U3","U4","U5","U6","U7","U8","D1","D2","D3","D4","D5","D6","D7","D8","D9"]
label_sequence = side

# 저장 경로가 없으면 생성
if not os.path.exists(save_path):
    os.makedirs(save_path)

# JSON 파일 목록 가져오기
json_files = glob.glob(json_path)

for json_file in tqdm(json_files, desc="Processing JSON files"):
    with open(json_file) as json_data:
        data = json.load(json_data)
        filename = data['imagePath']
        h, w = data['imageHeight'], data['imageWidth']
        
        # 저장할 텍스트 파일 경로
        save_anno = os.path.join(save_path, os.path.splitext(os.path.basename(filename))[0] + ".txt")
        
        keypoint_data = {label: (0,0,0) for label in label_sequence}
        cx, cy, width, height = 0, 0, 0, 0
        
        for shape in data['shapes']:
            label = shape['label']
            if label == 'side' or label == 'dog' or label == 'tooth':
                x1, y1 = shape['points'][0]
                x2, y2 = shape['points'][1]
                width = (x2-x1) / w
                height = (y2-y1) / h
                cx = (x1+(x2-x1)/2)/w
                cy = (y1+(y2-y1)/2)/h
            elif label in label_sequence:            
                x, y = shape['points'][0]
                x1 = x / w
                y1 = y / h
                category = shape.get('group_id', 2)  # group_id가 없으면 기본값 2 사용
                keypoint_data[label] = (x1, y1, category)
        
        yolo_annotation_line = f"0 {cx:.6f} {cy:.6f} {width:.6f} {height:.6f}"
        for label in label_sequence:
            x, y, visible = keypoint_data[label]
            yolo_annotation_line += f" {x:.6f} {y:.6f} {visible}"
        
        with open(save_anno, 'w') as f:
            f.write(yolo_annotation_line + "\n")

print("처리가 완료되었습니다.")

Processing JSON files: 100%|██████████| 50/50 [00:00<00:00, 6878.16it/s]

처리가 완료되었습니다.
